#### Install the necessary package.

In [1]:
!pip install watson-developer-cloud==1.5

#### We start by importing the following libraries.

In [2]:
import requests
import urllib.request
import time
import datetime
import csv
import re
from bs4 import BeautifulSoup

#### I've pre-defined some redundant functions here so as to reduce the complexity and size of the code

In [3]:
#function to check if we have access to the website
def access_to_server(a):
    if(access.status_code == 200):
        print("Access to PlaneCrashInfo.com: Granted")
    else:
        print("Access to PlaneCrashInfo.com: Denied")
        
#function to pause for a little so that the website server does not flag us as a spammer
def take_a_break():
    time.sleep(0.1)      #we pause for 3 seconds

#function to access the website
def access_website(url):
    access = None
    access = requests.get(url,timeout=10)
    return access   

def get_data(search_string, regex):
    pattern = re.compile(regex)
    matches = pattern.finditer(search_string)

    for match in matches:
        match = match.span()

    span = search_string[match[0]:match[1]]
    return span

#### Next, we set the url to the Plane Crash Info website and access the site with our requests library. We also set the base url to create a dictionary with years as the key.

In [4]:
base_url = "http://www.planecrashinfo.com"
access = requests.get('http://www.planecrashinfo.com/database.htm')
access_to_server(access)
source = BeautifulSoup(access.text, 'html.parser')

Access to PlaneCrashInfo.com: Granted


 #### Here we build a dictionary with the url stores as the value and the year as the key

In [5]:
a_tags = source.find_all("a")
years = {a.text.strip(): {"url": base_url + a["href"] if a["href"][0] == "/" else base_url + "/" + a["href"]} for a in a_tags if a.text.strip().isdigit()}

#### Here we pause for a little to avoid getting flagged as a spammer.

In [6]:
take_a_break()

#### Now we start collecting the unstructured data

In [7]:
done = 0
for year, database in years.items():       #first we loop for each year from 1920 until 2019
    print(year)
    take_a_break()
    response_year = access_website(database["url"])
    access_to_server(response_year)
    parser_year = BeautifulSoup(response_year.content, 'html.parser')
#    if done == 0:
#        break
    a_tags2 = parser_year.find_all("a")
    a_tags2 = [a for a in a_tags2 if "Return to Home" not in a.text]
    for a in a_tags2:

            # request to crash detail page
            response_crash = access_website(base_url + "/" + year + a["href"] if a["href"][0] == "/" else base_url + "/" + year + "/" + a["href"])
            parser_crash = BeautifulSoup(response_crash.content, 'html.parser')

            # get all table content except the first row(table title)
            tr_tags = parser_crash.find_all("tr")
            tr_tags = tr_tags[1:]

            database = [tr.find_all("td")[1].text.strip() for tr in tr_tags]
            # get all aboard
            aboard = database[9]
            all_aboard = get_data(aboard, r'^\d+|^\W')

            # get all passengers aboard
            passengers_aboard = get_data(aboard, r'(?<=\(passengers:)\d+|(?<=\(passengers:)\W')

            # get all crew aboard
            crew_aboard = get_data(
                aboard, r'(?<=crew:)\d+|(?<=crew:)\W')

            # get all fatalities
            fatalities = database[10]

            all_fatalities = get_data(fatalities, r'^\d+|^\W')

            # get all passengers fatalities
            passenger_fatalities = get_data(fatalities, r'(?<=\(passengers:)\d+|(?<=\(passengers:)\W')

            # get all crew fatalities
            crew_fatalities = get_data(fatalities, r'(?<=crew:)\d+|(?<=crew:)\W')

            # remove aboard info and store each value separately
            database.pop(9)
            database.insert(9, all_aboard)
            database.insert(10, passengers_aboard)
            database.insert(11, crew_aboard)

            # remove fatalities info and store each value separately
            database.pop(12)
            database.insert(12, all_fatalities)
            database.insert(13, passenger_fatalities)
            database.insert(14, crew_fatalities)
            take_a_break()

1920
Access to PlaneCrashInfo.com: Granted
1921
Access to PlaneCrashInfo.com: Granted
1922
Access to PlaneCrashInfo.com: Granted
1923
Access to PlaneCrashInfo.com: Granted
1924
Access to PlaneCrashInfo.com: Granted
1925
Access to PlaneCrashInfo.com: Granted
1926
Access to PlaneCrashInfo.com: Granted
1927
Access to PlaneCrashInfo.com: Granted
1928
Access to PlaneCrashInfo.com: Granted
1929
Access to PlaneCrashInfo.com: Granted
1930
Access to PlaneCrashInfo.com: Granted
1931
Access to PlaneCrashInfo.com: Granted
1932
Access to PlaneCrashInfo.com: Granted
1933
Access to PlaneCrashInfo.com: Granted
1934
Access to PlaneCrashInfo.com: Granted
1935
Access to PlaneCrashInfo.com: Granted
1936
Access to PlaneCrashInfo.com: Granted
1937
Access to PlaneCrashInfo.com: Granted
1938
Access to PlaneCrashInfo.com: Granted
1939
Access to PlaneCrashInfo.com: Granted
1940
Access to PlaneCrashInfo.com: Granted
1941
Access to PlaneCrashInfo.com: Granted
1942
Access to PlaneCrashInfo.com: Granted
1943
Access

IndexError: list index out of range

In [8]:
database

[]